In [2]:
import tensorflow as tf
import keras
import numpy as np

In [40]:
class ResBlock(keras.layers.Layer):
    def __init__(self,filters,do_upsample=False,name=None,**kwargs):
        super().__init__(name=None,**kwargs)
        self.conv_1 = keras.layers.SeparableConv2D(filters,3,padding="same")
        self.conv_2 = keras.layers.Conv2D(filters,3,padding="same")

        self.conv_residual = keras.layers.SeparableConv2D(filters,3,padding="same")
        self.upsample = tf.keras.layers.UpSampling2D(2)
        self.do_upsample = do_upsample

    def call(self,x):
        residual = x
        x = self.conv_1(x)
        x = self.conv_2(x)
        residual = self.conv_residual(residual)
        x = x + residual
        if self.do_upsample:
            x = self.upsample(x)
        return x


input = keras.layers.Input((512,512,3),batch_size=1)

filters_orig = 64
filters = filters_orig
x = ResBlock(filters)(input)
x0 = ResBlock(filters,do_upsample=True)(x)

filters = filters_orig*2
x = ResBlock(filters,do_upsample=False)(x0)
x = ResBlock(filters,do_upsample=True)(x)

filters = filters_orig
x = ResBlock(filters,do_upsample=False)(x) + tf.keras.layers.UpSampling2D(2)(x0)
x = ResBlock(3,do_upsample=False)(x)

model = keras.Model(input,x,name="super_resolution")
model.summary()

Model: "super_resolution"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_32 (InputLayer)          [(1, 512, 512, 3)]   0           []                               
                                                                                                  
 res_block_153 (ResBlock)       (1, 512, 512, 64)    37494       ['input_32[0][0]']               
                                                                                                  
 res_block_154 (ResBlock)       (1, 1024, 1024, 64)  46400       ['res_block_153[0][0]']          
                                                                                                  
 res_block_155 (ResBlock)       (1, 1024, 1024, 128  165376      ['res_block_154[0][0]']          
                                )                                                  

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpsvngp60d\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpsvngp60d\assets
